# 01 - Data Overview

This notebook provides an initial exploration of the enriched FIPE datasets. We'll examine:
- Dataset shape and memory usage
- Column types and basic statistics
- Missing values analysis
- Duplicate records check

## Datasets
- `fipe_cars_enriched.csv`: Historical FIPE data (~599k records)
- `fipe_2022_enriched.csv`: 2022 FIPE data subset (~290k records)


In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Add src to path to import our modules
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root / "src"))

from data.loader import DatasetLoader

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Set style for plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✅ Imports completed successfully")


✅ Imports completed successfully


## 1. Load Datasets


In [2]:
# Initialize loader
loader = DatasetLoader()

# Load both datasets
df_cars, df_2022 = loader.load_all()

print("\n" + "="*60)
print("Datasets loaded successfully!")
print("="*60)


Loading /app/data/processed/fipe_cars_enriched.csv...
Loaded 599,007 rows and 16 columns
Loading /app/data/processed/fipe_2022_enriched.csv...
Loaded 290,275 rows and 16 columns

Datasets loaded successfully!


## 2. Dataset Shape and Memory Usage


In [3]:
def print_dataset_info(name: str, df: pd.DataFrame):
    """Print shape and memory usage information for a dataset."""
    shape = df.shape
    memory_mb = df.memory_usage(deep=True).sum() / (1024 ** 2)
    
    print(f"\n{'='*60}")
    print(f"Dataset: {name}")
    print(f"{'='*60}")
    print(f"Shape: {shape[0]:,} rows × {shape[1]} columns")
    print(f"Memory usage: {memory_mb:.2f} MB")
    print(f"Memory per row: {memory_mb * 1024 / shape[0]:.2f} KB")
    print(f"\nColumns ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:2d}. {col}")

# Print info for both datasets
print_dataset_info("fipe_cars_enriched", df_cars)
print_dataset_info("fipe_2022_enriched", df_2022)



Dataset: fipe_cars_enriched
Shape: 599,007 rows × 16 columns
Memory usage: 381.33 MB
Memory per row: 0.65 KB

Columns (16):
   1. brand
   2. model
   3. year
   4. price
   5. km
   6. state
   7. city
   8. fuel_type
   9. transmission
  10. engine_size
  11. color
  12. doors
  13. condition
  14. age_years
  15. year_of_reference
  16. month_of_reference

Dataset: fipe_2022_enriched
Shape: 290,275 rows × 16 columns
Memory usage: 184.76 MB
Memory per row: 0.65 KB

Columns (16):
   1. brand
   2. model
   3. year
   4. price
   5. km
   6. state
   7. city
   8. fuel_type
   9. transmission
  10. engine_size
  11. color
  12. doors
  13. condition
  14. age_years
  15. year_of_reference
  16. month_of_reference


## 3. Column Types and Basic Statistics


In [4]:
# Data types
print("="*60)
print("Data Types - fipe_cars_enriched")
print("="*60)
print(df_cars.dtypes)
print("\n" + "="*60)
print("Data Types - fipe_2022_enriched")
print("="*60)
print(df_2022.dtypes)


Data Types - fipe_cars_enriched
brand                  object
model                  object
year                    int64
price                 float64
km                    float64
state                  object
city                   object
fuel_type              object
transmission           object
engine_size           float64
color                  object
doors                   int64
condition              object
age_years               int64
year_of_reference       int64
month_of_reference     object
dtype: object

Data Types - fipe_2022_enriched
brand                  object
model                  object
year                    int64
price                 float64
km                    float64
state                  object
city                   object
fuel_type              object
transmission           object
engine_size           float64
color                  object
doors                   int64
condition              object
age_years               int64
year_of_reference    

In [5]:
# Numerical statistics
print("="*60)
print("Numerical Statistics - fipe_cars_enriched")
print("="*60)
print(df_cars.describe())


Numerical Statistics - fipe_cars_enriched
           year      price        km  engine_size     doors  age_years  \
count 599007.00  599007.00 599007.00    599007.00 599007.00  599007.00   
mean    2008.75  111580.60 142955.81         2.27      4.00      12.81   
std        9.19  291223.74 102061.72         1.03      0.55       9.20   
min     1985.00    1679.00      0.00         0.70      2.00      -1.00   
25%     2001.00   20150.00  64275.50         1.60      4.00       5.00   
50%     2010.00   42989.00 123469.00         2.00      4.00      11.00   
75%     2016.00   96921.00 213162.00         2.80      4.00      20.00   
max     2023.00 8600000.00 559200.00         6.70      5.00      38.00   

       year_of_reference  
count          599007.00  
mean             2021.57  
std                 0.57  
min              2021.00  
25%              2021.00  
50%              2022.00  
75%              2022.00  
max              2023.00  


In [6]:
# Categorical statistics
print("="*60)
print("Categorical Statistics - fipe_cars_enriched")
print("="*60)
categorical_cols = df_cars.select_dtypes(include=['object']).columns
if len(categorical_cols) > 0:
    print(df_cars[categorical_cols].describe())
else:
    print("No categorical columns found.")


Categorical Statistics - fipe_cars_enriched
                  brand                   model   state       city fuel_type  \
count            599007                  599007  599007     599007    599007   
unique               87                    6171      27        131         3   
top     VW - VolksWagen  Buggy/M-8/M-8 Long 1.6      SP  São Paulo  Gasoline   
freq              58386                     898  138364      27822    493501   

       transmission   color condition month_of_reference  
count        599007  599007    599007             599007  
unique            2      10         4                 12  
top          manual  Branco       Bom            January  
freq         435066  149606    285996              71823  


In [7]:
# First few rows
print("="*60)
print("First 5 rows - fipe_cars_enriched")
print("="*60)
df_cars.head()


First 5 rows - fipe_cars_enriched


,brand,model,year,price,km,state,city,fuel_type,transmission,engine_size,color,doors,condition,age_years,year_of_reference,month_of_reference
0,Acura,NSX 3.0,1995,40374.00,290445.00,BA,Juazeiro,Gasoline,manual,3.00,Preto,4,Bom,26,2021,January
1,Acura,NSX 3.0,1994,38939.00,341185.00,PE,Jaboatão dos Guararapes,Gasoline,manual,3.00,Vermelho,4,Bom,27,2021,January
2,Acura,NSX 3.0,1993,37648.00,232996.00,MG,Juiz de Fora,Gasoline,manual,3.00,Branco,5,Bom,28,2021,January
3,Acura,NSX 3.0,1992,35962.00,259837.00,BA,Camaçari,Gasoline,manual,3.00,Prata,4,Regular,29,2021,January
4,Acura,NSX 3.0,1991,32863.00,362691.00,SP,São Paulo,Gasoline,manual,3.00,Branco,4,Bom,30,2021,January


## 4. Missing Values Analysis


In [8]:
def analyze_missing_values(name: str, df: pd.DataFrame):
    """Analyze and display missing values in a dataset."""
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    
    missing_df = pd.DataFrame({
        'Column': missing.index,
        'Missing Count': missing.values,
        'Missing Percentage': missing_pct.values
    })
    missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
    
    print(f"\n{'='*60}")
    print(f"Missing Values Analysis - {name}")
    print(f"{'='*60}")
    
    if len(missing_df) == 0:
        print("✅ No missing values found!")
    else:
        print(f"\nFound {len(missing_df)} columns with missing values:\n")
        print(missing_df.to_string(index=False))
        
        # Visualize missing values
        plt.figure(figsize=(10, 6))
        missing_df_sorted = missing_df.sort_values('Missing Percentage', ascending=True)
        plt.barh(missing_df_sorted['Column'], missing_df_sorted['Missing Percentage'])
        plt.xlabel('Missing Percentage (%)')
        plt.title(f'Missing Values by Column - {name}')
        plt.tight_layout()
        plt.show()
    
    return missing_df

# Analyze missing values for both datasets
missing_cars = analyze_missing_values("fipe_cars_enriched", df_cars)
missing_2022 = analyze_missing_values("fipe_2022_enriched", df_2022)



Missing Values Analysis - fipe_cars_enriched
✅ No missing values found!

Missing Values Analysis - fipe_2022_enriched
✅ No missing values found!


## 5. Duplicate Records Check


In [9]:
def analyze_duplicates(name: str, df: pd.DataFrame):
    """Analyze duplicate records in a dataset."""
    total_duplicates = df.duplicated().sum()
    duplicate_pct = (total_duplicates / len(df)) * 100
    
    print(f"\n{'='*60}")
    print(f"Duplicate Records Analysis - {name}")
    print(f"{'='*60}")
    print(f"Total records: {len(df):,}")
    print(f"Duplicate records: {total_duplicates:,} ({duplicate_pct:.2f}%)")
    print(f"Unique records: {len(df) - total_duplicates:,}")
    
    if total_duplicates > 0:
        print(f"\n⚠️  Found {total_duplicates:,} duplicate records!")
        print("\nFirst 10 duplicate records:")
        duplicates = df[df.duplicated(keep=False)].sort_values(by=list(df.columns))
        display(duplicates.head(10))
        
        # Check for duplicates based on key columns (excluding price which might vary)
        key_cols = ['brand', 'model', 'year', 'km', 'state', 'city', 
                   'fuel_type', 'transmission', 'engine_size', 'color', 
                   'doors', 'condition', 'age_years']
        key_duplicates = df.duplicated(subset=key_cols).sum()
        print(f"\nDuplicates based on key features (excluding price): {key_duplicates:,}")
    else:
        print("\n✅ No duplicate records found!")
    
    return total_duplicates

# Analyze duplicates for both datasets
dup_cars = analyze_duplicates("fipe_cars_enriched", df_cars)
dup_2022 = analyze_duplicates("fipe_2022_enriched", df_2022)



Duplicate Records Analysis - fipe_cars_enriched
Total records: 599,007
Duplicate records: 0 (0.00%)
Unique records: 599,007

✅ No duplicate records found!

Duplicate Records Analysis - fipe_2022_enriched
Total records: 290,275
Duplicate records: 0 (0.00%)
Unique records: 290,275

✅ No duplicate records found!


## 6. Summary and Key Findings


In [10]:
print("="*60)
print("SUMMARY - Initial Data Exploration")
print("="*60)

print("\n📊 Dataset Overview:")
print(f"  • fipe_cars_enriched: {df_cars.shape[0]:,} rows × {df_cars.shape[1]} columns")
print(f"  • fipe_2022_enriched: {df_2022.shape[0]:,} rows × {df_2022.shape[1]} columns")

print("\n💾 Memory Usage:")
memory_cars = df_cars.memory_usage(deep=True).sum() / (1024 ** 2)
memory_2022 = df_2022.memory_usage(deep=True).sum() / (1024 ** 2)
print(f"  • fipe_cars_enriched: {memory_cars:.2f} MB")
print(f"  • fipe_2022_enriched: {memory_2022:.2f} MB")
print(f"  • Total: {memory_cars + memory_2022:.2f} MB")

print("\n📋 Column Types:")
numeric_cols_cars = df_cars.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols_cars = df_cars.select_dtypes(include=['object']).columns.tolist()
print(f"  • Numerical columns: {len(numeric_cols_cars)}")
print(f"  • Categorical columns: {len(categorical_cols_cars)}")

print("\n❓ Missing Values:")
missing_cars_count = df_cars.isnull().sum().sum()
missing_2022_count = df_2022.isnull().sum().sum()
print(f"  • fipe_cars_enriched: {missing_cars_count} missing values")
print(f"  • fipe_2022_enriched: {missing_2022_count} missing values")

print("\n🔄 Duplicate Records:")
print(f"  • fipe_cars_enriched: {dup_cars:,} duplicates")
print(f"  • fipe_2022_enriched: {dup_2022:,} duplicates")

print("\n" + "="*60)
print("Next Steps:")
print("  → Proceed to notebook 02_target_analysis.ipynb")
print("  → Analyze price distribution and outliers")
print("  → Explore relationships between features and target")
print("="*60)


SUMMARY - Initial Data Exploration

📊 Dataset Overview:
  • fipe_cars_enriched: 599,007 rows × 16 columns
  • fipe_2022_enriched: 290,275 rows × 16 columns

💾 Memory Usage:
  • fipe_cars_enriched: 386.54 MB
  • fipe_2022_enriched: 187.29 MB
  • Total: 573.83 MB

📋 Column Types:
  • Numerical columns: 7
  • Categorical columns: 9

❓ Missing Values:
  • fipe_cars_enriched: 0 missing values
  • fipe_2022_enriched: 0 missing values

🔄 Duplicate Records:
  • fipe_cars_enriched: 0 duplicates
  • fipe_2022_enriched: 0 duplicates

Next Steps:
  → Proceed to notebook 02_target_analysis.ipynb
  → Analyze price distribution and outliers
  → Explore relationships between features and target
